In [1]:
from dataset import create_dataset, create_sampler, create_loader
from models.model_bbox import XVLM as XVLM_bbox
from models.model_captioning import XVLM as XVLM_caption

config = {'train_file': ['data/finetune/cosmos_caption_train.json'], 'val_file': ['data/finetune/cosmos_caption_val.json'], 
'test_file': ['data/finetune/cosmos_caption_val.json'], 'image_root': 'images/cosmos/', 
'vision_config': 'configs/config_swinB_384.json', 'use_clip_vit': False, 'use_swin': True, 
'image_res': 384, 'patch_size': 32, 'use_roberta': False, 'text_config': 'configs/config_bert.json', 
'text_encoder': 'data/bert-base-uncased', 'num_dec_layers': 6, 'batch_size_train': 5, 
'batch_size_test': 32, 'max_tokens': 40, 'label_smoothing': 0.1, 'max_length': 20, 'min_length': 5, 
'num_beams': 3, 'prompt': 'a picture of ', 'optimizer': {'opt': 'adamW', 'lr': 1e-05, 
'weight_decay': 0.01, 'lr_mult': 2}, 'schedular': {'sched': 'linear', 'lr': 1e-05, 'epochs': 5, 
'num_warmup_steps': 0.1}, 'start_eval': 0,'batch_size':5}
# train_dataset, val_dataset, test_dataset = create_dataset('cosmos', config);
# datasets = [train_dataset, val_dataset, test_dataset]


In [2]:
train_dataset, val_dataset, test_dataset = create_dataset('caption_cosmos', config);

/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:834: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:
train_dataset[0]

(tensor([[[-1.5441, -1.5149, -1.5149,  ..., -1.4127, -1.4127, -1.4127],
          [-1.5441, -1.5441, -1.5441,  ..., -1.4127, -1.3981, -1.3981],
          [-1.5733, -1.5587, -1.5879,  ..., -1.4127, -1.3981, -1.3981],
          ...,
          [-1.5149, -1.5149, -1.5149,  ..., -1.1207, -1.1207, -1.0769],
          [-1.5149, -1.5149, -1.5149,  ..., -1.1937, -1.0915, -1.0477],
          [-1.5149, -1.5149, -1.5149,  ..., -1.1937, -1.1791, -1.1353]],
 
         [[-1.5120, -1.5120, -1.5120,  ..., -1.3919, -1.3919, -1.3919],
          [-1.5120, -1.5120, -1.4820,  ..., -1.3769, -1.3769, -1.3769],
          [-1.4970, -1.4970, -1.4669,  ..., -1.3769, -1.3619, -1.3619],
          ...,
          [-1.7371, -1.7371, -1.7371,  ...,  1.2495,  1.2194,  1.2044],
          [-1.7371, -1.7371, -1.7371,  ...,  1.0994,  1.2645,  1.2945],
          [-1.7371, -1.7371, -1.7371,  ...,  0.9793,  1.1444,  1.2495]],
 
         [[-0.1720, -0.1293, -0.1435,  ...,  0.2404,  0.2404,  0.2404],
          [-0.1435, -0.1435,

In [6]:
train_loader, val_loader, test_loader = create_loader(datasets, [None, None, None], batch_size=[config['batch_size']] * 3,
                                                          num_workers=[4, 4, 4], is_trains=[True, False, False],
                                                          collate_fns=[None, None, None])

In [11]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
torch.cuda.set_device(2) 

In [8]:
model_caption = XVLM_caption(config=config)
model_caption.load_pretrained('coco_capt_cider_step_44275.th', config, is_eval=False)
model_caption = model_caption.to('cuda');

### Loading pretrained vision encoder
### Loading pretrained text encoder
load_capt_pretrain,  False
### Loading pretrained text encoder
load checkpoint from coco_capt_cider_step_44275.th
missing_keys:  []
unexpected_keys:  []


In [10]:
image = test_dataset[361][0][None,:]
image = image.to('cuda')
torch.manual_seed(42)
model_caption.generate(image,num_return_sequences=5,sample=True, num_beams=1,repetition_penalty=1.2)

(['a group of people sitting in front of a clock tower',
  'a group of people sitting on the square with a statue',
  'a group of people sitting on the plaza with a statue',
  'a group of people sitting on the square with a clock tower',
  'a group of people sitting on the steps with a statue'],
 tensor([-0.3649, -0.1935, -0.1850, -0.1832, -0.2098], device='cuda:2',
        grad_fn=<DivBackward0>))

In [12]:
torch.manual_seed(42)
device='cuda'
captions = []
for image, _, _, _ in tqdm(test_loader):
    image = image.to(device)
    c = model_caption.generate(image,num_return_sequences=3,sample=True, num_beams=1,repetition_penalty=1.5)
    cs = np.array_split(c[0],len(image))
    cs = [list(cs) for cs in cs]
    captions += cs

100%|██████████| 340/340 [05:18<00:00,  1.07it/s]


In [14]:
captions[212]

['a man in a leather jacket and a table',
 'a man in a leather jacket and a lady with a table',
 'a man in a leather jacket and a table']

In [ ]:
df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)